# ProWave - WaveNet-based Protein Generation

Authors: Hans Jakob Damsgaard & Lucas Balling

02456 Deep Learning project: ProGen

## Downloading Tape ##

In [1]:
!pip install tape_proteins

#### Importing Needed packages

In [2]:
import tape
import pip
from numpy import errstate,isneginf,array
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import torch
from torch.autograd import Variable
from torch.nn.parameter import Parameter
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.nn.init as init
# Matlab Plotting possibilities
import matplotlib.pyplot as plt

# Get Tape
from tape import TRRosetta
from tape.datasets import TRRosettaDataset

drive_path = "/Users/lucasballing/Desktop/DeepLearningProject/prowave-main/" #"C:/silabs"



### Download dataset

In [3]:
# Loading the pfam dataset and changing into a language model
!/Users/lucasballing/Desktop/DeepLearningProject/prowave-main/tape-master/download_data.sh -n
#LanguageModelingDataset(Dataset):

Do you wish to download and unzip the pretraining corpus? It is 7.7GB compressed and 19GB uncompressed? [y/n]^C


In [4]:
n

NameError: name 'n' is not defined